In [3]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [5]:
from PIL import Image
import os



In [10]:
# Define the root directory where your alphabet folders are located
root_directory = "D:\\SELFPROJECT\\archive\\asl_alphabet_train\\asl_alphabet_train\\digits"

# Define the target size
target_size = (200, 200)

# Define the output directory where the resized images will be saved
output_root_directory = "D:\\SELFPROJECT\\archive\\asl_alphabet_train"

# Create the output directory if it doesn't exist
if not os.path.exists(output_root_directory):
    os.makedirs(output_root_directory)

# Loop through the alphabet folders
for alphabet_folder in os.listdir(root_directory):
    alphabet_folder_path = os.path.join(root_directory, alphabet_folder)

    # Check if it's a directory (to exclude any non-folder files)
    if os.path.isdir(alphabet_folder_path):
        # Create a corresponding output folder
        output_alphabet_folder_path = os.path.join(output_root_directory, alphabet_folder)
        os.makedirs(output_alphabet_folder_path, exist_ok=True)

        # Loop through the images in the alphabet folder
        for filename in os.listdir(alphabet_folder_path):
            if filename.endswith('.jpg'):  # Adjust the file extension as needed
                img = Image.open(os.path.join(alphabet_folder_path, filename))
                resized_img = img.resize(target_size, Image.ANTIALIAS)

                # Save the resized image in the output folder
                output_filename = os.path.join(output_alphabet_folder_path, filename)
                resized_img.save(output_filename)


In [11]:
import os
import random
import shutil

# Define the source directory containing folders from 'b' to 'y' (excluding 'j')
source_directory = "D:\\SELFPROJECT\\archive\\asl_alphabet_test\\asl_alphabet_test"

# Define the destination directory to move the selected images
destination_directory = "D:\\SELFPROJECT\\archive\\asl_alphabet_test\\asl_alphabet_test"

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Loop through the folders in the source directory
for folder_name in os.listdir(source_directory):
    if folder_name != 'j' and os.path.isdir(os.path.join(source_directory, folder_name)):
        folder_path = os.path.join(source_directory, folder_name)

        # Get a list of all image files in the folder
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

        # Select 10 random images from the folder (or all available if there are fewer than 10)
        selected_images = random.sample(image_files, min(10, len(image_files)))

        # Rename and move the selected images to the destination directory
        for image_file in selected_images:
            new_name = f'{folder_name}_test_{image_file}'
            source_path = os.path.join(folder_path, image_file)
            destination_path = os.path.join(destination_directory, new_name)
            shutil.move(source_path, destination_path)

        # Delete the empty folder
        shutil.rmtree(folder_path)

print("Operation completed.")


Operation completed.


In [1]:
import os
import shutil

# Define the source folder where all your images are located
source_folder = "D:\\SELFPROJECT\\archive\\asl_alphabet_test\\asl_alphabet_test"

# Create a list of category names (A-Z and 4 others)
categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
              'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
              'nothing', 'unknown', 'del', 'space']

# Loop through each category and create a folder for it
for category in categories:
    folder_path = os.path.join(source_folder, category)
    os.makedirs(folder_path, exist_ok=True)

# Loop through each image file in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith(".jpg"):  # Adjust the file extension as needed
        # Extract the category from the filename (assuming the category is in the filename)
        category = filename[0]  # Change this extraction logic as needed

        # Construct the source and destination paths
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(source_folder, category, filename)

        # Move the image to its respective category folder
        shutil.move(source_path, destination_path)

print("Images organized into category folders.")


Images organized into category folders.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define constants
image_size = (200, 200)
batch_size = 32
num_classes = len(os.listdir("D:\\SELFPROJECT\\archive\\asl_alphabet_train\\asl_alphabet_train"))  # Number of gesture classes

# Data Augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2  # Adjust the split ratio as needed
)

# Load and preprocess the dataset
train_generator = datagen.flow_from_directory(
    "D:\\SELFPROJECT\\archive\\asl_alphabet_train\\asl_alphabet_train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    "D:\\SELFPROJECT\\archive\\asl_alphabet_train\\asl_alphabet_train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Create the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20  # Adjust the number of epochs as needed 
)

# Evaluate the model on test data
test_data = keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(
    "D:\\SELFPROJECT\\archive\\asl_alphabet_test\\asl_alphabet_test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test accuracy: {test_acc}")

# Save the model for future use
model.save('gesture_recognition_model.h5')
